In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.3 MB/s eta 0:00

In [ ]:
 !nvidia-smi

Sat May  6 04:06:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
from datasets import load_dataset, DatasetDict, load_from_disk
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertModel, GPT2Tokenizer
from tqdm import tqdm
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# train_dataset = load_dataset(path="wikitext", name="wikitext-103-raw-v1", split="train")
# dev_dataset = load_dataset(path="wikitext", name="wikitext-103-raw-v1", split="validation")

# load trimed datasets
from datasets import load_dataset, DatasetDict, Dataset

train_dataset = Dataset.load_from_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_train')
dev_dataset = Dataset.load_from_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_dev')

In [ ]:
batch_size = 256
progress_bar = tqdm(range(len(dev_dataset)//batch_size))
dev_loader = DataLoader(dataset=dev_dataset, batch_size=batch_size, shuffle=False)
embeddings = []
for k in dev_loader:
  inputs = tokenizer(k['text'], return_tensors='pt', padding='max_length', truncation=True, max_length=512)
  inputs = {k:v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
  for i in torch.mean(outputs['last_hidden_state'], dim=1):
    embeddings.append(i.cpu().numpy())
  progress_bar.update(1)
np.save('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_dev_embedding.npy', np.array(embeddings, dtype='float16'), allow_pickle=True)

4it [00:09,  1.82s/it]                       

In [ ]:
torch.cuda.empty_cache()

In [ ]:
batch_size = 512
progress_bar = tqdm(range(len(train_dataset)//batch_size))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
embeddings = []
for j, k in enumerate(train_loader):
  inputs = tokenizer(k['text'], return_tensors='pt', padding='max_length', truncation=True, max_length=512)
  inputs = {k:v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
  for i in torch.mean(outputs['last_hidden_state'], dim=1):
    embeddings.append(i.cpu().numpy())
  if j % 100 == 99:
    np.save(f'/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_bert_embedding/train_embedding{j-99}-{j}.npy', np.array(embeddings, dtype='float16'), allow_pickle=True)
    embeddings = []
  progress_bar.update(1)
np.save(f'/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_bert_embedding/train_embedding{j-(j % 100)}-{j}.npy', np.array(embeddings, dtype='float16'), allow_pickle=True)
print('saved')

 12%|█▏        | 99/810 [1:00:33<7:14:51, 36.70s/it]
811it [47:07,  2.98s/it]                         

saved


In [ ]:
dev_dataset = Dataset.load_from_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_dev')
embeddings = np.load('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_dev_embedding.npy')
print(embeddings.shape)
dev_dataset.add_column('embedding', [i.flatten() for i in embeddings]).save_to_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_embed_dev')

(880, 768)


Saving the dataset (0/1 shards):   0%|          | 0/880 [00:00<?, ? examples/s]

In [ ]:
dev_dataset = Dataset.load_from_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_embed_dev')
dev_dataset

Dataset({
    features: ['text', 'embedding'],
    num_rows: 880
})

In [ ]:
embeddings = np.load('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_bert_embedding/train_embedding0-99.npy')
for i in range(1, 8):
  embeddings = np.concatenate((embeddings, np.load(f'/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_bert_embedding/train_embedding{i}00-{i}99.npy')))
embeddings = np.concatenate((embeddings, np.load(f'/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_bert_embedding/train_embedding800-810.npy')))
print(embeddings.shape)
train_dataset.add_column('embedding', [i.flatten() for i in embeddings]).save_to_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_embed_train')

(414971, 768)


Saving the dataset (0/3 shards):   0%|          | 0/414971 [00:00<?, ? examples/s]

In [ ]:
dev_dataset = Dataset.load_from_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_embed_dev')
dev_dataset

Dataset({
    features: ['text', 'embedding'],
    num_rows: 880
})

In [ ]:
embeddings = dev_dataset['embedding']
X = np.stack(embeddings)
X.shape

(880, 768)

In [ ]:
from sklearn.decomposition import PCA
pca_128 = PCA(n_components=128)
pca_128.fit(X)
X_128 = pca_128.transform(X)

In [ ]:
dev_dataset.add_column('embedding_pca', [i.flatten() for i in X_128]).save_to_disk('/content/gdrive/MyDrive/ColabNotebooks/SSNLP/final project/trimed_wiki103_embed_dev_pca')

Saving the dataset (0/1 shards):   0%|          | 0/880 [00:00<?, ? examples/s]